# Hyperparameter Tuning using HyperDrive

## Azure ML imports

In [16]:
import logging
import os
import csv
import pkg_resources
import joblib

import numpy as np
import pandas as pd

from matplotlib import pyplot as plt

from sklearn import datasets
from sklearn.metrics import confusion_matrix

import azureml.core
from azureml.core.compute import AmlCompute
from azureml.core.environment import Environment
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.core.dataset import Dataset
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.sklearn import SKLearn
from azureml.widgets import RunDetails

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.57.0


## Initialize workspace

Initialize a workspace object from persisted configuration.

In [17]:
ws = Workspace.from_config()

print(
    'Workspace name: ' + ws.name, 
    'Azure region: ' + ws.location, 
    'Subscription id: ' + ws.subscription_id, 
    'Resource group: ' + ws.resource_group, sep = '\n'
    )

Workspace name: quick-starts-ws-275688
Azure region: westeurope
Subscription id: cdbe0b43-92a0-4715-838a-f2648cc7ad21
Resource group: aml-quickstarts-275688


## Create an Azure HyperDrive experiment

Let's create an experiment named `heart-failure-hd-exp` and a folder to hold the training scripts. The script runs will be recorded under the experiment in Azure.

The best practice is to use separate folders for scripts and its dependent files for each step and specify that folder as the `source_directory for the step. This helps reduce the size of the snapshot created for the step (only the specific folder is snapshotted). Since changes in any files in the source_directory would trigger a re-upload of the snapshot, this helps keep the reuse of the step when there are no changes in the source_directory of the step.

In [18]:
# Choose a name for the run history container in the workspace
experiment_name = 'heart-failure-hd-experiment'
project_folder = './Capstone-Project'

experiment = Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
heart-failure-hd-experiment,quick-starts-ws-275688,Link to Azure Machine Learning studio,Link to Documentation


### Create or attach an AmlCompute cluster

You will need to create a [compute target](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#compute-target) for your HyperDrive run.

In [19]:
# Choose a name for your CPU cluster
compute_cluster_name = "atul-trdigi-compute"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=compute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(
        vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
        #vm_priority = 'lowpriority', # optional
        min_nodes=0,
        max_nodes=5)
    compute_target = ComputeTarget.create(ws, compute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

# For a more detailed view of current AmlCompute status, use get_status()
print(compute_target.get_status().serialize())

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2025-03-17T14:44:04.465000+00:00', 'errors': None, 'creationTime': '2025-03-17T09:47:14.647405+00:00', 'modifiedTime': '2025-03-17T09:47:24.616560+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 5, 'nodeIdleTimeBeforeScaleDown': 'PT1800S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


## Dataset

The data is loaded into the workspace using `TabularDataFactory` in the `train.py` script.

## Hyperdrive Configuration

For the HyperDrive experiment, we chose the [`LogisticRegression`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) classifier from scikit-learn.

The script `train.py` takes care of data collection, cleansing and splitting, model training and testing. Hyperparameter sampling and applying the early stopping policy is performed by HyperDrive.

### Data collection, cleansing and splitting

The dataset is loaded using `TabularDatasetFactory`. The cleansing process drops rows with empty values and performs one hot encoding for categorical columns (our dataset does not have any). The dataset is split into train and test sets. 70% of the data is used for training and 30% for testing.

### Hyperparameter sampling

The project uses two hyperparameters:

- `--C`: inverse regularization strength
- `--max_iter`: maximum iteration to converge for the scikit-learn Logistic Regression model

I use [random parameter sampling](https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.hyperdrive.randomparametersampling?view=azure-ml-py). Random sampling supports discrete and continuous hyperparameters. It supports early termination of low-performance runs. In random sampling, hyperparameter values are randomly selected from the defined search space. Random parameter sampling is good approach for discovery learning as well as hyperparameter combinations.

### Model training and testing

Model training and testing is performed using scikit-learn's Logistical Regression model. In `train.py`, metrics are generated and logged. The accuracy is used to benchmark the model.

### Applying early stopping policy

The execution of the pipeline is stopped if the conditions specified by the policy are met.

The model uses [BanditPolicy](https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.hyperdrive.banditpolicy?view=azure-ml-py).

Bandit policy is based on slack factor/slack amount and evaluation interval. Bandit ends runs when the primary metric isn't within the specified slack factor/slack amount of the most successful run.

See [HyperDriveConfig Class](https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.hyperdrive.hyperdriveconfig?view=azure-ml-py) for a complete list of configuration parameters.


In [21]:
env = Environment.get(workspace=ws,name = 'Azure-ML-Tutorial')

In [25]:
# Early termination policy (not required if using Bayesian sampling)
early_termination_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

# Params that you will be using during training
param_sampling = RandomParameterSampling({
    "--C": uniform(0.001, 100),
    "--max_iter": choice(10, 50, 100, 150, 200)
    })

# Training directory and script
train_dir = "./hyperdrive-training-script"
train_script = "train.py"

from azureml.core import ScriptRunConfig
# SKLearn estimator for use with train.py
estimator = ScriptRunConfig(
    source_directory=train_dir,
    script=train_script,
    compute_target=compute_cluster_name,
    environment=env
    )

# HyperDriveConfig using the estimator, hyperparameter sampler, and policy
hyperdrive_run_config = HyperDriveConfig(
    run_config=estimator,
    hyperparameter_sampling=param_sampling,
    primary_metric_name='Accuracy',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=25,
    max_concurrent_runs=5,
    policy=early_termination_policy,
    )

In [26]:
# Submit your experiment
hyperdrive_run = experiment.submit(hyperdrive_run_config)

## Run Details

Use the `RunDetails` widget to show the different experiments.

In [27]:
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)
hyperdrive_run

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_9be5a193-f55c-44d1-81df-ddf19cc0ba2e
Web View: https://ml.azure.com/runs/HD_9be5a193-f55c-44d1-81df-ddf19cc0ba2e?wsid=/subscriptions/cdbe0b43-92a0-4715-838a-f2648cc7ad21/resourcegroups/aml-quickstarts-275688/workspaces/quick-starts-ws-275688&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2025-03-17T15:31:44.1578899Z][GENERATOR][DEBUG]Sampled 5 jobs from search space 
[2025-03-17T15:31:44.5041317Z][SCHEDULER][INFO]Scheduling job, id='HD_9be5a193-f55c-44d1-81df-ddf19cc0ba2e_0' 
[2025-03-17T15:31:44.6337272Z][SCHEDULER][INFO]Scheduling job, id='HD_9be5a193-f55c-44d1-81df-ddf19cc0ba2e_3' 
[2025-03-17T15:31:44.6229319Z][SCHEDULER][INFO]Scheduling job, id='HD_9be5a193-f55c-44d1-81df-ddf19cc0ba2e_1' 
[2025-03-17T15:31:44.6131796Z][SCHEDULER][INFO]Scheduling job, id='HD_9be5a193-f55c-44d1-81df-ddf19cc0ba2e_2' 
[2025-03-17T15:31:45.1096896Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_9be5a193-f55c-44d1-81df-ddf19cc0ba2e_2' 
[2025-03-17T15:3

ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "Execution failed. User process 'python' exited with status code 1. Please check log file 'user_logs/std_log.txt' for error details. Error: Traceback (most recent call last):\n  File \"train.py\", line 3, in <module>\n    import joblib\nModuleNotFoundError: No module named 'joblib'\n\n Marking the experiment as failed because initial child jobs have failed due to user error",
        "messageParameters": {},
        "details": []
    },
    "time": "0001-01-01T00:00:00.000Z"
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Activity Failed:\n{\n    \"error\": {\n        \"code\": \"UserError\",\n        \"message\": \"Execution failed. User process 'python' exited with status code 1. Please check log file 'user_logs/std_log.txt' for error details. Error: Traceback (most recent call last):\\n  File \\\"train.py\\\", line 

## Best Model

Get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
# Get your best run
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(f"Best run arguments: {best_run.get_details()['runDefinition']['arguments']}")
print(f"Best run metrics: {best_run.get_metrics()}")
print(f"Best run file names: {best_run.get_file_names()}")

In [ ]:
# Save the best model
joblib.dump(value=best_run.id, filename="./outputs/hyperdrive_model.joblib")

## Model Deployment

As AutoML produced the better model, the HyperDrive model will not be deployed.